In [ ]:
# import bags of words 

In [38]:
import requests, re
from bs4 import BeautifulSoup

news_num = 6

class Feed:
    
    def __init__(self, name, url):
        self.name = name
        self.url = url
    
    def makeSoup(self):
        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup    
    

    def getHeaders(self):
        
        soup = self.makeSoup()
        
        if self.name == 'Onet':
            divs = soup.find_all(class_="boxContent")
            articles_sets = [div.find_all('a') for div in divs]
            onet_dic = {'name' : self.name}     
            for articles in articles_sets:
                for article in articles:                    
                    onet_dic[article.find(class_="title").text.strip()] = article.attrs.get('href')
            return onet_dic   
        
        
        if self.name == 'Wirtualna Polska':
            divs = soup.find_all(class_="sc-1fu2hk8-0 jIlknD")
            articles_sets = [div.find_all('a') for div in divs]
            wp_dic = {'name' : self.name}         
            for articles in articles_sets:
                for article in articles:                
                    if len(wp_dic) <= news_num+1:
                        try:
                            wp_dic[article.find_all('div')[1].text.strip()] = article.attrs.get('href')
                        except IndexError:
                            wp_dic[article.find('div').text.strip()] = article.attrs.get('href')
                    else:
                        return wp_dic
                    
        
        if self.name == 'W Polityce':
            divs = soup.find_all(class_="nu-lead-articles") + soup.find_all(class_="nu-tile-container nu-main-col--5")
            articles_sets = [div.find_all('a') for div in divs]
            wpolityce_dic = {'name' : self.name}
            for articles in articles_sets:
                for article in articles:
                    if len(wpolityce_dic) <= news_num+1:
                        wpolityce_dic[article.find(class_="short-title").text.strip()] = str(self.url[:-1])+str(article.attrs.get('href'))
                    else:
                        return wpolityce_dic
                    
                    
        if self.name == 'Przeglad Sportowy':
            div_main = soup.find(class_='wdgBests boxWrapper')
            div_second = soup.find(class_='items')
            div_stream = soup.find(class_='items solrList')
            przegladsportowy_dic = {'name' : self.name}
            
            for a in div_main.find_all('a'):
                przegladsportowy_dic[a.h3.text.strip()] = a.attrs.get('href')
                
            for a in div_stream.find_all('a'):
                przegladsportowy_dic[a.div.h3.text.strip()] = a.attrs.get('href')
                
            for a in div_second.find_all('a'):
                przegladsportowy_dic[a.div.h3.text.strip()] = a.attrs.get('href')
                
            return przegladsportowy_dic
        
        
        if self.name == 'Onet Sport':
            div_main = soup.find(class_='wdgBests boxWrapper')
            div_second = soup.find(class_='items')
            div_stream = soup.find(class_='items solrList')
            onetsport_dic = {'name' : self.name}
            
            for a in div_main.find_all('a'):
                onetsport_dic[a.h3.text.strip()] = a.attrs.get('href')
                
            for a in div_stream.find_all('a'):
                onetsport_dic[a.div.h3.text.strip()] = a.attrs.get('href')
                
            for a in div_second.find_all('a'):
                onetsport_dic[a.div.h3.text.strip()] = a.attrs.get('href')
                
            return onetsport_dic


        
onet = Feed('Onet', 'https://www.onet.pl/')
onetsport = Feed('Onet Sport', 'https://sport.onet.pl/')
wp = Feed('Wirtualna Polska', 'https://www.wp.pl/')
wpolityce = Feed('W Polityce', 'https://wpolityce.pl/')
przegladsportowy = Feed('Przeglad Sportowy', 'https://www.przegladsportowy.pl/')


feed_list = [onet, wp, wpolityce, przegladsportowy, onetsport]

def writePrzegladSportowy():

    with open('sport_headers.txt', 'a+')as f:
        actual_list = f.readlines()
        for k in przegladsportowy.getHeaders().keys():
            if k != 'name' and k not in actual_list:
                f.write(k+'\n')

def writeOnetSport():

    with open('sport_headers.txt', 'a+')as f:
        actual_list = f.readlines()
        for k in onetsport.getHeaders().keys():
            if k != 'name' and k not in actual_list:
                f.write(k+'\n')

with open('all_headers.txt', 'a+') as f:
    actual_list = f.readlines()
    for feed in feed_list:
        for k in feed.getHeaders().keys():
            if k != 'name' and k not in actual_list:
                    f.write(k+'\n')
        
    


extract key words from article headlines

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
import pandas as pd
import re

with open('polish.stopwords.txt') as f:
    polish_sw = [word.replace('\n','') for word in f.readlines()]
    
with open('sport_headers.txt') as f:
    docs = f.read().split('\n')[:-1]

    

def pre_process(text):
    
    text=text.lower()
    text=re.sub("<!--?.*?-->","",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text


def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 

def extract_topn_from_vector(feature_names, sorted_items, topn=10):

    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results



docs = list(map(lambda x:pre_process(x), docs))



cv=CountVectorizer(max_df=0.85,stop_words=polish_sw)
word_count_vector=cv.fit_transform(docs)


tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()

 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
# #extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,11)

with open('all_headers.txt', 'r') as f:
    all_headers = f.readlines()

vectorized_headers = dict.fromkeys(all_headers, 0)

print(vectorized_headers)
    

 

{'Szczyt G7. Policja starła się z protestującymi. W ruch poszły armatki wodne i gaz łzawiący\n': 0, 'Wojna o francuskie wino? Unia odpowie Trumpowi\n': 0, 'Incydent na konwencji Lewicy, mamy wideo\n': 0, 'Bardzo poważna sytuacja. Jasne stanowisko Tuska\n': 0, 'Ogień ogarnia nie tylko Amazonię. Alarm z Afryki\n': 0, 'Czy dało się uniknąć dramatu w Tatrach?\n': 0, 'Setki nowych ognisk. Wojsko rusza do walki\n': 0, 'Konwencja Lewicy. "Ziobro przed Trybunał Stanu"\n': 0, 'Pierwsze przestępstwo w kosmosie. Śledztwo NASA\n': 0, 'Niemcy przygotowują się na katastrofę atomową\n': 0, '"Gazeta Polska" organizuje koncert w Nowym Jorku. Znany pianista odmawia wystąpienia\n': 0, 'Czarzasty ostro o Ziobrze. "Świństwogenny"\n': 0, '7-latka zabrała tacie kluczyki i ruszyła w podróż\n': 0, 'Marsz Równości w Gorzowie Wlkp. Zdjęcia\n': 0, 'Belgijski książę popadł w ostry spór z państwem\n': 0, 'Tusk skomentował udział Rosji w szczycie G7\n': 0, 'Utrudnienia na podmiejskiej linii średnicowej\n': 0, 'Jaka 